In [1]:
import os
import torchaudio
import pandas as pd
from tqdm import tqdm
import sounddevice as sd

In [2]:
def play_audio_segment(audio_tensor, sample_rate):
    # audio_tensor shape: [channels, samples]
    audio_np = audio_tensor.numpy().T  # Trasponi per shape (samples, channels)
    sd.play(audio_np, sample_rate)
    sd.wait()

def label_speakers(audio_path, transcript_path):
    df = pd.read_csv(transcript_path, sep="\t")
    waveform, sample_rate = torchaudio.load(audio_path)
    if df['speaker'].str.lower().isin({'ellie', 'participant', 'ignore'}).any():
        print(f"{transcript_path} Labeling already done, skipping.")
        return
    speaker_labels = {}
    speakers = df['speaker'].unique()

    for spk in speakers:
        # Trova il segmento più lungo per questo speaker
        spk_segments = df[df['speaker'] == spk].copy()
        spk_segments["duration"] = spk_segments["stop_time"] - spk_segments["start_time"]
        longest_seg = spk_segments.sort_values("duration", ascending=False).iloc[0]

        start_sample = int(longest_seg['start_time'] * sample_rate)
        end_sample = int(longest_seg['stop_time'] * sample_rate)
        segment_audio = waveform[:, start_sample:end_sample]

        print(f"\nSpeaker: {spk} - playing **longest** segment from {start_sample/sample_rate:.2f}s to {end_sample/sample_rate:.2f}s "
              f"({longest_seg['duration']:.2f}s long)")
        play_audio_segment(segment_audio, sample_rate)
        choice = input("Label this speaker as (E)llie, (P)articipant, (O)ther: ").strip().lower()
        if choice == 'e':
            speaker_labels[spk] = "Ellie"
        elif choice == 'p':
            speaker_labels[spk] = "Participant"
        else:
            speaker_labels[spk] = "ignore"

    df['speaker'] = df['speaker'].map(speaker_labels)
    df.to_csv(transcript_path, sep="\t", index=False)
    print(f"Updated transcript saved to {transcript_path}")

In [3]:
edaic_dir = "../datasets/EDAIC-WOZ"

# Process all sessions for speaker labeling
sessions = sorted([d for d in os.listdir(edaic_dir) if os.path.isdir(os.path.join(edaic_dir, d))])

for session in tqdm(sessions, desc="Labeling speakers"):
    session_path = os.path.join(edaic_dir, session)
    base_name = session.split("_")[0]
    audio_path = os.path.join(session_path, f"{base_name}_AUDIO.wav")
    transcript_path = os.path.join(session_path, f"{base_name}_TRANSCRIPT.csv")
    
    if os.path.exists(audio_path) and os.path.exists(transcript_path):
        print(f"\n=== Processing session {session} ===")
        try:
            label_speakers(audio_path, transcript_path)
        except Exception as e:
            print(f"Error processing {session}: {e}")
            continue
    else:
        print(f"Skipping {session}: missing audio or transcript file")

Labeling speakers:   0%|          | 0/30 [00:00<?, ?it/s]


=== Processing session 602_P ===
../datasets/EDAIC-WOZ\602_P\602_TRANSCRIPT.csv Labeling already done, skipping.

=== Processing session 604_P ===
../datasets/EDAIC-WOZ\604_P\604_TRANSCRIPT.csv Labeling already done, skipping.

=== Processing session 617_P ===
../datasets/EDAIC-WOZ\617_P\617_TRANSCRIPT.csv Labeling already done, skipping.

=== Processing session 624_P ===
../datasets/EDAIC-WOZ\624_P\624_TRANSCRIPT.csv Labeling already done, skipping.

=== Processing session 633_P ===


Labeling speakers:  17%|█▋        | 5/30 [00:00<00:01, 18.57it/s]

../datasets/EDAIC-WOZ\633_P\633_TRANSCRIPT.csv Labeling already done, skipping.

=== Processing session 636_P ===
../datasets/EDAIC-WOZ\636_P\636_TRANSCRIPT.csv Labeling already done, skipping.

=== Processing session 637_P ===


Labeling speakers:  30%|███       | 9/30 [00:00<00:01, 14.41it/s]

...

Updated transcript saved to ../datasets/EDAIC-WOZ\705_P\705_TRANSCRIPT.csv

=== Processing session 709_P ===

Speaker: SPEAKER_02 - playing **longest** segment from 0.03s to 58.55s (58.52s long)

Speaker: SPEAKER_00 - playing **longest** segment from 93.50s to 123.07s (29.57s long)

Speaker: SPEAKER_01 - playing **longest** segment from 449.44s to 452.78s (3.34s long)


Labeling speakers:  93%|█████████▎| 28/30 [23:40<02:51, 85.68s/it]

Updated transcript saved to ../datasets/EDAIC-WOZ\709_P\709_TRANSCRIPT.csv

=== Processing session 716_P ===

Speaker: SPEAKER_01 - playing **longest** segment from 30.39s to 50.09s (19.70s long)

Speaker: SPEAKER_02 - playing **longest** segment from 60.90s to 86.86s (25.97s long)

Speaker: SPEAKER_00 - playing **longest** segment from 180.58s to 198.87s (18.29s long)


Labeling speakers: 100%|██████████| 30/30 [24:52<00:00, 49.74s/it]

Updated transcript saved to ../datasets/EDAIC-WOZ\716_P\716_TRANSCRIPT.csv
Skipping temp_results: missing audio or transcript file


In [4]:
daic_dir = "../datasets/DAIC-WOZ" 

sessions = ["318_P", "321_P", "341_P", "362_P"] # https://github.com/adbailey1/daic_woz_process/tree/master

for session in tqdm(sessions, desc="Labeling speakers"):
    session_path = os.path.join(daic_dir, session)
    base_name = session.split("_")[0]
    audio_path = os.path.join(session_path, f"{base_name}_AUDIO.wav")
    transcript_path = os.path.join(session_path, f"{base_name}_TRANSCRIPT.csv")
    
    if os.path.exists(audio_path) and os.path.exists(transcript_path):
        print(f"\n=== Processing session {session} ===")
        try:
            label_speakers(audio_path, transcript_path)
        except Exception as e:
            print(f"Error processing {session}: {e}")
            continue
    else:
        print(f"Skipping {session}: missing audio or transcript file")

Labeling speakers:   0%|          | 0/4 [00:00<?, ?it/s]


=== Processing session 318_P ===

Speaker: SPEAKER_00 - playing **longest** segment from 9.61s to 17.49s (7.89s long)

Speaker: SPEAKER_01 - playing **longest** segment from 391.36s to 412.69s (21.33s long)


Labeling speakers:  25%|██▌       | 1/4 [01:17<03:52, 77.64s/it]

Updated transcript saved to ../datasets/DAIC-WOZ\318_P\318_TRANSCRIPT.csv

=== Processing session 321_P ===

Speaker: SPEAKER_01 - playing **longest** segment from 11.08s to 17.87s (6.79s long)

Speaker: SPEAKER_00 - playing **longest** segment from 326.49s to 357.14s (30.65s long)

Speaker: SPEAKER_02 - playing **longest** segment from 37.85s to 65.64s (27.79s long)


Labeling speakers:  50%|█████     | 2/4 [04:02<04:17, 128.94s/it]

Updated transcript saved to ../datasets/DAIC-WOZ\321_P\321_TRANSCRIPT.csv

=== Processing session 341_P ===

Speaker: SPEAKER_01 - playing **longest** segment from 9.12s to 14.30s (5.19s long)

Speaker: SPEAKER_00 - playing **longest** segment from 385.29s to 405.64s (20.35s long)

Speaker: SPEAKER_02 - playing **longest** segment from 68.31s to 73.19s (4.89s long)


Labeling speakers:  75%|███████▌  | 3/4 [13:39<05:33, 333.52s/it]

Updated transcript saved to ../datasets/DAIC-WOZ\341_P\341_TRANSCRIPT.csv

=== Processing session 362_P ===

Speaker: SPEAKER_01 - playing **longest** segment from 479.98s to 491.53s (11.56s long)

Speaker: SPEAKER_00 - playing **longest** segment from 552.00s to 560.80s (8.79s long)


Labeling speakers: 100%|██████████| 4/4 [14:43<00:00, 220.82s/it]

Updated transcript saved to ../datasets/DAIC-WOZ\362_P\362_TRANSCRIPT.csv
